In [1]:
import sys
import os

# Determine the root directory (one level up from the notebook directory)
root_dir = os.path.abspath('..')

# Add the root directory to sys.path
sys.path.append(root_dir)

In [2]:
import pandas as pd
from sqlalchemy import create_engine, text
from sqlmodel import Session
from core.db import engine

def query_onchain_transaction_history(method_id: str, to_address: str) -> pd.DataFrame:
    with Session(engine) as session:
        # Execute the query
        query = text(f"""
        SELECT * FROM public.onchain_transaction_history
        WHERE method_id = :method_id AND to_address = lower(:to_address)
        """)  # Wrap the query with text()
        result = session.execute(query, {"method_id": method_id, "to_address": to_address})
        
        # Convert the result to a DataFrame
        df = pd.DataFrame(result.fetchall(), columns=result.keys())
        
    return df

df = query_onchain_transaction_history('0x71b8dc69', '0xC71BA0E3C1FB9CBcB15fbC677e78C99aC1bc590B')
df.head()

,id,tx_hash,block_number,from_address,to_address,method_id,input,value,timestamp,chain
0,6a5fd0af-34ba-4e81-b1db-e7be9652d362,0xe9142c8249d8f56b3c28134afb6018eba9102182f5c9...,252370321,0xbc05da14287317fe12b1a2b5a0e1d756ff1801aa,0xc71ba0e3c1fb9cbcb15fbc677e78c99ac1bc590b,0x71b8dc69,0x71b8dc69000000000000000000000000355ec27c9d45...,0.0,1726050496,arbitrum_one
1,e6d3936c-28f2-425d-acff-c547c7de35e8,0xdb888b623e399f437a2e93812ac38a872ddba48eb6d2...,252369949,0xbc05da14287317fe12b1a2b5a0e1d756ff1801aa,0xc71ba0e3c1fb9cbcb15fbc677e78c99ac1bc590b,0x71b8dc69,0x71b8dc69000000000000000000000000355ec27c9d45...,0.0,1726050402,arbitrum_one
2,8ee8dccd-4ea1-4b7b-bd25-975036993a5d,0xfa898588a756c8af957e42247415b4aa81793e762abe...,254128317,0x78e90b46ee542441e9fdcd1e072c3bc962fbd62d,0xc71ba0e3c1fb9cbcb15fbc677e78c99ac1bc590b,0x71b8dc69,0x71b8dc69000000000000000000000000355ec27c9d45...,0.0,1726493547,arbitrum_one
3,511f3e5d-bdfb-4b84-b55b-44c7c09297d8,0x1a759f45ff700005b6830601b21d2a07bfe8508e74cd...,252619272,0x658e36f00b397ec7aaef9f465fb05e1aec9a8363,0xc71ba0e3c1fb9cbcb15fbc677e78c99ac1bc590b,0x71b8dc69,0x71b8dc69000000000000000000000000355ec27c9d45...,0.0,1726113166,arbitrum_one
4,70fa2b80-6797-4147-b3e6-985022bfd557,0xb839bfad5182c0efda4d7524cea392ff23e18d2272e3...,254194456,0x34f59297ad78a54ed80f4ebeaa7bdb250221ba5c,0xc71ba0e3c1fb9cbcb15fbc677e78c99ac1bc590b,0x71b8dc69,0x71b8dc69000000000000000000000000355ec27c9d45...,0.0,1726510202,arbitrum_one


In [3]:
from web3 import AsyncWeb3, Web3
from web3.eth import Contract
from datetime import datetime


import json


def read_abi(token: str):
    with open(f"../config/{token.lower()}_abi.json") as f:
        data = json.load(f)
        return data

w3 = Web3(Web3.HTTPProvider("https://bitter-wandering-feather.arbitrum-mainnet.quiknode.pro/862a558ad28be94cf6b1ccae509bdca74a19086a"))

rockonyx_delta_neutral_vault_abi = read_abi("rockonyxdeltaneutralvault")
vault_contract = w3.eth.contract(
    address="0xC71BA0E3C1FB9CBcB15fbC677e78C99aC1bc590B",
    abi=rockonyx_delta_neutral_vault_abi,
)


In [7]:
from web3 import Web3
from core.config import settings

def _extract_pendle_event(entry):
    # Parse the account parameter from the topics field
    from_address = None
    if len(entry["topics"]) >= 2:
        from_address = f'0x{entry["topics"][1].hex()[26:]}'  # For deposit event

    # token_in = None
    # if len(entry["topics"]) >= 3:
    #     token_in = f'0x{entry["topics"][2].hex()[26:]}'

    # Parse the amount and shares parameters from the data field
    data = entry["data"].hex()

    if entry["topics"][0].hex() == settings.PENDLE_COMPLETE_WITHDRAW_EVENT_TOPIC:
        pt_amount = int(data[2:66], 16) / 1e18
        sc_amount = int(data[66 : 66 + 64], 16) / 1e6
        shares = int(data[66 + 64 : 66 + 2 * 64], 16) / 1e6
        total_amount = int(data[66 + 2 * 64 : 66 + 3 * 64], 16) / 1e6
        eth_amount = 0
    else:
        pt_amount = int(data[2:66], 16) / 1e18
        eth_amount = int(data[66 : 66 + 64], 16) / 1e18
        sc_amount = int(data[66 + 64 : 66 + 2 * 64], 16) / 1e6
        total_amount = int(data[66 + 64 * 2 : 66 + 3 * 64], 16) / 1e6
        shares = int(data[66 + 3 * 64 : 66 + 4 * 64], 16) / 1e6

    return pt_amount, eth_amount, sc_amount, total_amount, shares, from_address

def get_logs_from_tx_hash(tx_hash: str, topic: str = None) -> list:
    # Connect to the Ethereum node
    web3 = Web3(Web3.HTTPProvider(settings.ARBITRUM_MAINNET_INFURA_URL))
    
    # Check if the connection is successful
    if not web3.is_connected():
        raise ConnectionError("Failed to connect to the Ethereum node.")
    
    # Get the transaction receipt
    tx_receipt = web3.eth.get_transaction_receipt(tx_hash)
    
    if not tx_receipt:
        raise ValueError(f"No transaction found for hash: {tx_hash}")
    
    # Extract logs from the transaction receipt
    logs = tx_receipt['logs']
    
    # Filter logs by topic if provided
    if topic is not None:
        logs = [log for log in logs if log['topics'][0].hex() == topic]
    
    return logs

def get_shares(user: str, decimals = 1e6):
    tvl = vault_contract.functions.balanceOf(user).call()

    return tvl / decimals


logs = get_logs_from_tx_hash(df['tx_hash'].iloc[0], topic='0xf943cf10ef4d1e3239f4716ddecdf546e8ba8ab0e41deafd9a71a99936827e45')
pt_amount, eth_amount, sc_amount, total_amount, shares, from_address = _extract_pendle_event(logs[0])


In [9]:
import pandas as pd
import time

# Assuming df is your existing DataFrame
results = []

for index, row in df.iterrows():
    tx_hash = row['tx_hash']  # Get the transaction hash from the current row
    from_address = row['from_address']  # Get the from_address from the current row
    
    # Get logs for the transaction hash
    logs = get_logs_from_tx_hash(tx_hash, topic='0xf943cf10ef4d1e3239f4716ddecdf546e8ba8ab0e41deafd9a71a99936827e45')
    
    if logs:  # Check if logs are available
        # Extract event data from the first log
        pt_amount, eth_amount, sc_amount, total_amount, shares, from_address_event = _extract_pendle_event(logs[0])
        
        # Get current shares using the get_shares function
        current_shares = get_shares(Web3.to_checksum_address(from_address))  # Assuming you want to use from_address from the event
        
        # Append the results to the list
        results.append({
            'from_address': from_address,
            'init_shares': shares,
            'current_shares': current_shares
        })
        time.sleep(0.05)

# Create a new DataFrame from the results
new_df = pd.DataFrame(results)

new_df

,from_address,init_shares,current_shares
0,0xbc05da14287317fe12b1a2b5a0e1d756ff1801aa,46.250822,0.000000
1,0xbc05da14287317fe12b1a2b5a0e1d756ff1801aa,46.270600,0.000000
2,0x78e90b46ee542441e9fdcd1e072c3bc962fbd62d,183.290850,183.290850
3,0x658e36f00b397ec7aaef9f465fb05e1aec9a8363,17106.101446,19931.405583
4,0x34f59297ad78a54ed80f4ebeaa7bdb250221ba5c,91.495735,91.495735
...,...,...,...
164,0x7b397f33fd8083a96acc1e41247e322b0374def0,2937.029209,0.000000
165,0xee11037211a4840669a67c773959d233e5398e5d,935.332530,0.000000
166,0xee11037211a4840669a67c773959d233e5398e5d,66.997137,0.000000
167,0x392ab1aac3f9adc9aa56761ab9d92ee9aebaa92b,2688.490421,0.000000


In [16]:
# Deduplicate 'from_address' in new_df and keep the last occurrence
new_df = new_df.drop_duplicates(subset='from_address', keep='last')

# Display the deduplicated DataFrame
new_df

,from_address,init_shares,current_shares
2,0x78e90b46ee542441e9fdcd1e072c3bc962fbd62d,183.290850,183.290850
4,0x34f59297ad78a54ed80f4ebeaa7bdb250221ba5c,91.495735,91.495735
5,0x9b5ae497fd1bf885a40d6f99d0c57245761e053c,18818.461366,18818.461366
6,0x2d5b62dcc466c6f1c0f3c25d0b689e7d75963cfb,139.306678,139.306678
9,0xfdf39d0789ef9965b63d176a0d41f21b46e3408b,48.815345,0.000000
...,...,...,...
163,0xbb78bd00b2f12c8263ec27a9872c028a2e48990e,4782.849747,0.000000
164,0x7b397f33fd8083a96acc1e41247e322b0374def0,2937.029209,0.000000
166,0xee11037211a4840669a67c773959d233e5398e5d,66.997137,0.000000
167,0x392ab1aac3f9adc9aa56761ab9d92ee9aebaa92b,2688.490421,0.000000


In [18]:
new_df.to_csv('all depositors pendle 26dec24.csv', index=False)

In [19]:
current_total_shares = new_df['current_shares'].sum()
current_total_shares

87926.51421